In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/THDL

[Errno 2] No such file or directory: '/content/drive/MyDrive/THDL'
/content


In [ ]:
# !git clone https://github.com/namhoai1811/IT5420_20222.git

Cloning into 'IT5420_20222'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 124 (delta 50), reused 108 (delta 34), pack-reused 0
Receiving objects: 100% (124/124), 23.88 MiB | 8.62 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Updating files: 100% (46/46), done.


In [3]:
cd IT5420_20222/

[Errno 2] No such file or directory: 'IT5420_20222/'
/content


In [4]:
pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import numpy,datetime,re
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
tentative_engagements = []
free_bds = []
corresponacy1 = []

In [ ]:
def matchingtablebyheaders(tbl1,tbl2):      
    # For matching
    Pref=np.zeros((len(tbl1),len(tbl2)))
        #assign the matching ratio between tbl1 and tbl2
    for i in range(len(tbl1)):
        for j in range(len(tbl2)):
            Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
    return Pref

In [ ]:
def listofpreferencespertable(tbl1,tbl2,Pref):
        #for chotot
    ipref_tom=np.zeros((len(tbl1),len(tbl2)),dtype=np.int8)
    #for bds
    ipref_bds=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
    pref_bds=np.zeros((len(tbl2),len(tbl1)),dtype=np.int8)
    preferred_rankings_data_1 = {}
    preferred_rankings_data_2 = {}
    #create list of preferred_rankings_chotot with header
    #assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl2 (17)
    for i in range(len(tbl2)):
        ipref_tom[:,i]=np.argsort(Pref[:,i])
        ipref_tom[:,i]=ipref_tom[::-1,i]
        preferred_rankings_data_1[i] = ipref_tom[:,i]
        #pref_tom[:,i]=np.sort(Pref[:,i])
        #pref_tom[:,i]=pref_tom[::-1,i]
    #ipref_tom

    #create list of preferred_rankings_bds with header
    #assign the array indexs of Pref sorted by the actual values decending based on the lenght of tbl1 (13)
    for i in range(len(tbl1)):
        #get indexs values of rows and store them in the rows as well
        ipref_bds[:,i] = np.argsort(Pref[i,:])
        ipref_bds[:,i] = ipref_bds[::-1,i]
        preferred_rankings_data_2[i] = ipref_bds[:,i]
        #pref_bds[:,i] = np.sort(Pref[i,:])
      #  pref_bds[:,i] = pref_bds[::-1,i]
    #ipref_bds
    return preferred_rankings_data_2,preferred_rankings_data_1

In [ ]:
#Imdb which still need to propose and get accepted successfully
#Keep track of the headers that "may" end up together
def init_free_bds(preferred_rankings_imdb):
    for imdb in preferred_rankings_imdb.keys():
        free_bds.append(imdb)

In [ ]:
def begin_matching(bds,preferred_rankings_data_1,preferred_rankings_data_2):
    #  '''Find the first free Tomaten available to a bds at any given time'''
    #print(preferred_rankings_data_1[bds])
    for tomate in preferred_rankings_data_1[bds]:
        #Boolean for whether tomate is taken or not
        taken_match = [couple for couple in tentative_engagements if tomate == couple [:][1]]
        if (len(taken_match) == 0):
            #tentatively engage the man and woman
            tentative_engagements.append([bds, tomate])
            free_bds.remove(bds)
            break
        elif (len(taken_match) > 0):

            current_bds = list(preferred_rankings_data_2[tomate].tolist()).index(taken_match[0][0])
            potential_bds = list(preferred_rankings_data_2[tomate].tolist()).index(bds)

            if (current_bds < potential_bds):
                ii=[]
             
            else:         
                free_bds.remove(bds)
                #The old bds is now single
                free_bds.append(taken_match[0][0])
                #Update the fiance of the tomaten (tentatively)
                taken_match[0][0] = bds
                break

In [ ]:
def stable_matching(preferred_rankings_data_1,preferred_rankings_data_2):
    '''Matching algorithm until stable match terminates'''
    while (len(free_bds) > 0):
        # print((free_bds))
        for imdb in free_bds:
            
            begin_matching(imdb,preferred_rankings_data_1,preferred_rankings_data_2)
        # print((free_bds))
        # break

In [ ]:
def check_short_matching(COL1,COL2):
    rate = 0.0
    if str(COL1) == 'nan' or str(COL2) == 'nan' :
        return 0
    COL1 = str(COL1)
    COL2 = str(COL2)
    sw = (fuzz.SequenceMatcher(None,COL1, COL2).ratio())
    return sw #/(max(len(COL1),len(COL2))*1.0)


In [ ]:
def check_match(st1,st2):
    str1 = np.array((st1.values).astype(str))
    str2 = np.array((st2.values).astype(str))
    # ret = np.zeros((len(str1),len(str2)))
    ret = np.zeros((len(str1)))
    
    for i in range(len(str1)):
        tmp = -1
        for j in range(len(str2)):
            # ret[i,j] = check_short_matching(str1[i],str2[j])
            tmp = max(tmp, check_short_matching(str1[i],str2[j]))
        ret[i] = tmp
    return numpy.average(ret)

In [ ]:
def matchingtablebyvalues(DATA_1_DF_SAMPLE,DATA_2_DF_SAMPLE,tbl1,tbl2):
    dd = []
    ret = []
    value1 = []
    value2 = []
    index_i = 0
    Pref=np.zeros((len(tbl1),len(tbl2)))
    for i in DATA_1_DF_SAMPLE:
        dd = []
        index_j = 0
        for j in DATA_2_DF_SAMPLE:
            table1 = DATA_1_DF_SAMPLE[i]
            table2 = DATA_2_DF_SAMPLE[j]
            value1 = (table1.values).astype(str)
            value2 = (table2.values).astype(str)
            #dd.append([ 100* (check_match(table1, table2)),i,j])
            Pref[index_i,index_j] = 100 * (np.float(check_match(table1, table2)))
            index_j+=1
        index_i+=1
    return Pref

In [ ]:
def outputforvalues(tbl1,tbl2,Pref):
    i = 0
    for item in tentative_engagements:
        matching=Pref[item[0],item[1]]
        print(tbl1[item[0]] + " -----" ,tbl2[item[1]],':',matching)

# alonhadat vs batdonsan123

In [9]:
alonhadat = pd.read_csv("/content/drive/MyDrive/H3T/IT5420_20222/data-standard/alonhadat.csv", index_col=0)


In [10]:
alonhadat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068 entries, 0 to 2377
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bedroom            2068 non-null   object 
 1   code               2067 non-null   float64
 2   date               2067 non-null   object 
 3   description        2068 non-null   object 
 4   diningroom         2068 non-null   object 
 5   direct             2067 non-null   object 
 6   floor              2068 non-null   object 
 7   introduce_contact  2068 non-null   object 
 8   juridical          2067 non-null   object 
 9   kitchen            2068 non-null   object 
 10  length             2067 non-null   object 
 11  link_image         2068 non-null   object 
 12  name_contact       2067 non-null   object 
 13  parking            2068 non-null   object 
 14  phone_contact      2067 non-null   object 
 15  price              2068 non-null   int64  
 16  project            2068 

In [ ]:
alonhadat = pd.DataFrame(alonhadat[['title', 'description', 'price', 'square', 'type', 'name_contact', 'phone_contact', 'date', 'direct', 'district', 'province', 'street', 'ward', 'floor', 'juridical', 'bedroom', 'length', 'width', 'link_image', 'url_page', 'kitchen', 'parking', 'terrace']])

In [13]:
batdongsan123 = pd.read_csv("/content/drive/MyDrive/H3T/IT5420_20222/data-standard/batdongsan123.csv", index_col=0)

In [14]:
batdongsan123['length'] = '0 m'
batdongsan123['width'] = '0 m'

In [ ]:
batdongsan123.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7833 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   acreage        7833 non-null   object 
 1   bathroom       7833 non-null   object 
 2   bedroom        7833 non-null   object 
 3   code           7833 non-null   int64  
 4   date           7833 non-null   object 
 5   description    7833 non-null   object 
 6   direction      7833 non-null   object 
 7   link_image     7598 non-null   object 
 8   name_contact   7831 non-null   object 
 9   phone_contact  246 non-null    float64
 10  price          7833 non-null   float64
 11  project        7833 non-null   object 
 12  title          7833 non-null   object 
 13  url_page       7833 non-null   object 
 14  ward           7833 non-null   object 
 15  district       7833 non-null   object 
 16  province       7833 non-null   object 
 17  street         7833 non-null   object 
 18  type    

In [ ]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    #add data
    # DATA_SAMPLE_1,tbl1,DATA_SAMPLE_2,tbl2=importdatasets()
    DATA_SAMPLE_1 = alonhadat.sample(50).astype(str)

    header_1 = list(alonhadat.columns.values)
    tbl1 = np.array(header_1)

    DATA_SAMPLE_2 = batdongsan123.sample(50).astype(str)

    header_2 = list(batdongsan123.columns.values)
    tbl2 = np.array(header_2)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(DATA_SAMPLE_1,DATA_SAMPLE_2,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)

    
    
main()

<ipython-input-8-f71726be21e7>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
<ipython-input-15-c247a0c48378>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

title ----- title : 70.9825186558915
price ----- price : 93.1961415396198
type ----- type : 100.0
phone_contact ----- phone_contact : 50.0
direct ----- direction : 87.36309523809524
province ----- province : 94.47919799498747
ward ----- ward : 64.2599061148092
juridical ----- juridical : 91.48496240601503
length ----- length : 87.48571428571428
link_image ----- link_image : 67.67379074965652
kitchen ----- kitchen : 61.5
terrace ----- terrace : 77.2
description ----- description : 57.08645804315947
name_contact ----- name_contact : 79.44419695511064
district ----- district : 92.69991645781118
floor ----- floor : 88.28571428571428
width ----- width : 84.3047619047619
parking ----- parking : 59.0
square ----- acreage : 68.46060606060607
street ----- street : 65.6414154043411
url_page ----- url_page : 76.50825878831128
date ----- date : 95.1
bedroom ----- bedroom : 89.80454545454545


In [ ]:
batdongsan123 = batdongsan123.rename(columns={'acreage': 'square'})
batdongsan123 = batdongsan123.rename(columns={'direction': 'direct'})

In [ ]:
batdongsan123 = pd.DataFrame(batdongsan123[['title', 'description', 'price', 'square', 'type', 'name_contact', 'phone_contact', 'date', 'direct', 'district', 'province', 'street', 'ward', 'floor', 'juridical', 'bedroom', 'length', 'width', 'link_image', 'url_page', 'kitchen', 'parking', 'terrace']])

# alonhadat vs ibatdongsan

In [16]:
ibatdongsan = pd.read_csv("/content/drive/MyDrive/H3T/IT5420_20222/data-standard/ibatdongsan.csv", index_col=0)

In [ ]:
ibatdongsan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7882 entries, 0 to 9994
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   area           7882 non-null   object
 1   bedroom        7882 non-null   object
 2   brief          7882 non-null   object
 3   code           7882 non-null   int64 
 4   date           7882 non-null   object
 5   diningroom     7882 non-null   object
 6   direct         7882 non-null   object
 7   floor          7882 non-null   object
 8   juridical      7882 non-null   object
 9   kitchen        7882 non-null   object
 10  length         7882 non-null   object
 11  link_image     7882 non-null   object
 12  name_contact   7882 non-null   object
 13  parking        7882 non-null   object
 14  phone_contact  7882 non-null   object
 15  price          7882 non-null   int64 
 16  terrace        7882 non-null   object
 17  title          7882 non-null   object
 18  type           7882 non-null

In [ ]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    #add data
    # DATA_SAMPLE_1,tbl1,DATA_SAMPLE_2,tbl2=importdatasets()
    DATA_SAMPLE_1 = alonhadat.sample(50).astype(str)

    header_1 = list(alonhadat.columns.values)
    tbl1 = np.array(header_1)

    DATA_SAMPLE_2 = ibatdongsan.sample(50).astype(str)

    header_2 = list(ibatdongsan.columns.values)
    tbl2 = np.array(header_2)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(DATA_SAMPLE_1,DATA_SAMPLE_2,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)

    
    
main()

<ipython-input-8-f71726be21e7>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
<ipython-input-15-c247a0c48378>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

title ----- title : 71.56627154916286
price ----- price : 96.53013784461152
type ----- type : 100.0
phone_contact ----- phone_contact : 81.07333333333334
direct ----- direct : 99.14545454545456
province ----- province : 94.32478745884319
ward ----- ward : 83.12231150708672
juridical ----- juridical : 94.87854251012146
length ----- length : 95.98809523809524
link_image ----- link_image : 77.88207058865201
kitchen ----- kitchen : 100.0
terrace ----- terrace : 100.0
district ----- district : 91.42832722832722
name_contact ----- name_contact : 78.34086538309333
floor ----- floor : 97.85714285714286
width ----- width : 95.86666666666666
parking ----- parking : 100.0
square ----- area : 62.25
street ----- street : 82.89767364324102
url_page ----- url_page : 78.16272187133453
date ----- date : 99.1
description ----- brief : 21.055631534202597
bedroom ----- bedroom : 99.41666666666667


In [ ]:
ibatdongsan = ibatdongsan.rename(columns={'area': 'square'})
ibatdongsan = ibatdongsan.rename(columns={'brief': 'description'})

In [ ]:
ibatdongsan = pd.DataFrame(ibatdongsan[['title', 'description', 'price', 'square', 'type', 'name_contact', 'phone_contact', 'date', 'direct', 'district', 'province', 'street', 'ward', 'floor', 'juridical', 'bedroom', 'length', 'width', 'link_image', 'url_page', 'kitchen', 'parking', 'terrace']])

# alonhadat vs nhadat24h

In [18]:
nhadat24h = pd.read_csv("/content/drive/MyDrive/H3T/IT5420_20222/data-standard/nhadat24h.csv", index_col=0)

In [ ]:
nhadat24h.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1551 entries, 0 to 1963
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bathroom          1551 non-null   object 
 1   bedroom           1551 non-null   object 
 2   code              1551 non-null   object 
 3   date              0 non-null      float64
 4   description       1551 non-null   object 
 5   direct            372 non-null    object 
 6   floor             1551 non-null   object 
 7   ground_area       1551 non-null   object 
 8   juridical         1551 non-null   object 
 9   kitchen           1551 non-null   object 
 10  length            1551 non-null   object 
 11  link_image        1551 non-null   object 
 12  livingroom        1551 non-null   object 
 13  name_project      1551 non-null   object 
 14  parking           1551 non-null   object 
 15  price             1551 non-null   int64  
 16  road_width        1551 non-null   object 


In [ ]:
nhadat24h['name_contact'] = np.nan
nhadat24h['phone_contact'] = np.nan

In [ ]:
def main():
    tentative_engagements.clear()
    free_bds.clear()
    #add data
    # DATA_SAMPLE_1,tbl1,DATA_SAMPLE_2,tbl2=importdatasets()
    DATA_SAMPLE_1 = alonhadat.sample(50).astype(str)

    header_1 = list(alonhadat.columns.values)
    tbl1 = np.array(header_1)

    DATA_SAMPLE_2 = nhadat24h.sample(50).astype(str)

    header_2 = list(nhadat24h.columns.values)
    tbl2 = np.array(header_2)

    #creation templates
    Pref_header=matchingtablebyheaders(tbl1,tbl2)
    Pref_values=matchingtablebyvalues(DATA_SAMPLE_1,DATA_SAMPLE_2,tbl1,tbl2)
    Pref = (0.5*Pref_header + 0.5*Pref_values)
    preferred_rankings_imdb,preferred_rankings_tomatoes=listofpreferencespertable(tbl1,tbl2,Pref)
    # print(preferred_rankings_tomatoes)
    init_free_bds(preferred_rankings_imdb)
    stable_matching(preferred_rankings_imdb,preferred_rankings_tomatoes)
    # print(tentative_engagements)
    outputforvalues(tbl1,tbl2,Pref)

    
    
main()

<ipython-input-8-f71726be21e7>:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Pref[i,j] = np.int(100*SequenceMatcher(None,tbl1[i].lower(),tbl2[j].lower()).ratio())
<ipython-input-15-c247a0c48378>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

title ----- title : 71.05315004378903
price ----- price : 95.85772765246449
type ----- type : 100.0
phone_contact ----- phone_contact : 50.0
district ----- district : 94.75520619731145
province ----- province : 91.23806139718107
ward ----- ward : 82.97125202798577
juridical ----- juridical : 84.42857142857142
length ----- length : 83.4047619047619
link_image ----- link_image : 77.22508874795896
kitchen ----- kitchen : 93.49090909090908
terrace ----- terrace : 93.6
description ----- description : 63.05020175114043
name_contact ----- name_contact : 50.0
floor ----- floor : 91.85714285714286
width ----- width : 83.16666666666666
parking ----- parking : 93.49090909090908
square ----- ground_area : 69.12020202020202
street ----- street : 80.49595985057303
url_page ----- url_page : 76.95558062402586
date ----- date : 50.0
direct ----- direct : 66.68333333333334
bedroom ----- bedroom : 99.77777777777777


In [ ]:
nhadat24h = nhadat24h.rename(columns={'ground_area': 'square'})

In [ ]:
nhadat24h = pd.DataFrame(nhadat24h[['title', 'description', 'price', 'square', 'type', 'name_contact', 'phone_contact', 'date', 'direct', 'district', 'province', 'street', 'ward', 'floor', 'juridical', 'bedroom', 'length', 'width', 'link_image', 'url_page', 'kitchen', 'parking', 'terrace']])

# Concat data to one table

In [ ]:
dataset = pd.concat([alonhadat, batdongsan123, ibatdongsan, nhadat24h]) 

In [ ]:
dataset = dataset.reset_index(drop=True)

In [ ]:
dataset

,title,description,price,square,type,name_contact,phone_contact,date,direct,district,...,floor,juridical,bedroom,length,width,link_image,url_page,kitchen,parking,terrace
0,"Cần Bán Căn Tập Thể Nguyễn Trãi, 80m Chỉ 2,15 ...",TẬP THỂ NGUYỄN TRÃI - LÔ GÓC - 3 MẶT THOÁNG - ...,2.150000e+09,80 m,Cần bán căn hộ chung cư,Vi Thị Bích,0968.004.685,15/05/2023,None,Thanh Xuân,...,4 t,Sổ hồng/ Sổ đỏ,3 pn,0m,0m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/can-ban-can-tap-the-n...,Có,Có,Không
1,"Cắt lỗ 150 triệu - Bán gấp căn 2PN FPT, 68m2 g...","Cắt lỗ 150 triệu - Bán gấp căn 2PN FPT, giá 1,...",1.550000e+09,68 m,Cần bán căn hộ chung cư,Châu Tường Vân,0355.653.937,14/05/2023,None,Ngũ Hành Sơn,...,25 t,Sổ hồng/ Sổ đỏ,2 pn,0m,0m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/cat-lo-150-trieu-ban-...,Không,Không,Không
2,"Chính chủ bán căn chung cư Khương Đình, Thanh ...",Tôi chính chủ cần bán căn hộ chung cư tại Khươ...,7.000000e+08,40 m,Cần bán căn hộ chung cư,Gia Quyền,0933.646.568,06/05/2023,None,Thanh Xuân,...,8 t,Sổ hồng/ Sổ đỏ,1 pn,10m,4m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/chinh-chu-ban-can-chu...,Có,Không,Có
3,Giá cực sốc!!! Cả thành phố duy nhất chỉ có mộ...,Chính chủ bán chung cư giá cực rẻ căn chung cư...,1.490000e+09,69 m,Cần bán căn hộ chung cư,Đinh Nguyệt,0983.430.986,09/05/2023,None,Đông Anh,...,0 t,Sổ đỏ,2 pn,0m,0m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/gia-cuc-soc-ca-thanh-...,Không,Không,Không
4,"NHANH THÌ CÒN bán chung cư mini phố Trần Hòa,5...",Rất tiếc mà phải bán căn chung cư mini nằm ở p...,1.100000e+09,50 m,Cần bán căn hộ chung cư,Vi Quốc Tuấn,0377.469.168,10/05/2023,None,Hoàng Mai,...,8 t,Sổ hồng/ Sổ đỏ,2 pn,0m,0m,https://alonhadat.com.vn/files/properties/2023...,https://alonhadat.com.vn/nhanh-thi-con-ban-chu...,Có,Không,Không
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19329,CĂN HỘ LAVITA THUẬN AN 1TY8 2PN 70M2 0904609771,📍Chính sách bán hàng:,1.800000e+09,70 m,Cần bán căn hộ chung cư,NaN,NaN,NaN,NaN,None,...,2 t,Hợp đồng mua bán,15 pn,10 m,7 m,https://nhadat24h.net.vn/storage/product/7205/...,https://nhadat24h.net.vn/du-an/can-ho-lavita-t...,Có,Không,Không
19330,"Tumys Phú Mỹ, 29,9tr/m2, ưu đãi 100 KH hỗ trợ ...",Dự án Tumys Home Phú Mỹ đang triển khai giai đ...,1.500000e+09,50 m,Cần bán căn hộ chung cư,NaN,NaN,NaN,NaN,Tân Thành,...,2 t,Sổ Hồng,25 pn,0 m,0 m,https://nhadat24h.net.vn/storage/product/7016/...,https://nhadat24h.net.vn/du-an/tumys-phu-my-29...,Có,Không,Không
19331,Chính Chủ Cần Bán Nhanh CĂN NHÀ Vị Trí Đẹp tại...,None,2.350000e+09,65 m,Cần bán căn hộ chung cư,NaN,NaN,NaN,NaN,12,...,2 t,Sổ Đỏ,0 pn,0 m,0 m,https://nhadat24h.net.vn/storage/product/1678/...,https://nhadat24h.net.vn/du-an/chinh-chu-can-b...,Có,Không,Không
19332,CHÍNH CHỦ BÁN CĂN HỘ VIEW ĐẸP TẠI CHUNG CƯ TỪ...,None,1.200000e+09,70 m,Cần bán căn hộ chung cư,NaN,NaN,NaN,NaN,None,...,2 t,Sổ Đỏ,0 pn,0 m,0 m,https://nhadat24h.net.vn/storage/product/4826/...,https://nhadat24h.net.vn/du-an/chinh-chu-ban-c...,Có,Không,Không


In [ ]:
dataset.to_csv("/content/drive/MyDrive/THDL/IT5420_20222/all_dataset_after_schema-matching.csv", index=False, header=True)